In [ ]:
import json
from pathlib import Path

# 加载文件
with open("zero_shot_merged_1.json", "r", encoding="utf-8") as f:
    zero_shot_data = json.load(f)

with open("cleaned_results.json", "r", encoding="utf-8") as f:
    cleaned_results = json.load(f)

# 创建一个 dict 便于按 sent_id 查询 zero_shot 中的预测结果
zero_shot_dict = {item["sent_id"]: item for item in zero_shot_data}

# 定义有效字符级细分类别
valid_fine_types = {"缺字漏字", "错别字错误", "缺少标点", "错用标点"}

# 处理并生成新文件内容
merged_results = []
for entry in cleaned_results:
    sent_id = entry["sent_id"]
    if sent_id in zero_shot_dict:
        zero_pred = zero_shot_dict[sent_id]
        
        # 合并字符级 coarse 预测
        pred_cg = set(entry.get("pred_CourseGrainedErrorType", []))
        if "字符级错误" in zero_pred.get("pred_CourseGrainedErrorType", []):
            pred_cg.add("字符级错误")
        entry["pred_CourseGrainedErrorType"] = list(pred_cg)
        
        # 合并字符级 fine 预测
        pred_fg = set(entry.get("pred_FineGrainedErrorType", []))
        for err_type in zero_pred.get("pred_FineGrainedErrorType", []):
            if err_type in valid_fine_types:
                pred_fg.add(err_type)
        entry["pred_FineGrainedErrorType"] = list(pred_fg)

    merged_results.append(entry)

# 保存为新文件
output_path = "merged_cleaned_results_with_char_errors.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(merged_results, f, ensure_ascii=False, indent=2)

output_path
